In [100]:
#######
## feb 11, 2022
## testing the functions zach wrote for data download
## doesn't use wget and fully "pythonized" - uses requests instead
## using asf api to generate URLS


# import libraries
import numpy as np
import re
import zipfile
import getpass
import os  # for chdir, getcwd, path.basename, path.exists
import pandas as pd # for DatetimeIndex
import netrc
import glob
import requests
from shapely.geometry import Polygon, mapping
from datetime import datetime
from subprocess import PIPE, Popen
from os.path import join, isdir, isfile, basename
import progressbar
from tqdm import tqdm
import logging
import asf_search as asf

# for map to generate wkt
from folium import Map
from folium.plugins import Draw
import geopandas as gpd

# import the convert_image function from convert.py
from genericpath import isfile
import sys
sys.path.append('../')

from uavsar_pytools.convert import convert_image
from uavsar_pytools.download import download_image

log = logging.getLogger(__name__)
logging.basicConfig()
log.setLevel(logging.WARNING)

In [101]:
# input NASA Earthdata credentials here
NASA_USER = input("Enter Username: ")
NASA_PASS = getpass.getpass("Enter Password: ")

Enter Username:  jacktarricone
Enter Password:  ············


### Create Folium map to generate at WKT file

The ```asf_search``` package uses the Well-known text or [WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) files to indentify areas of interest to search thier database. The [Folium](https://python-visualization.github.io/folium/#:~:text=folium%20makes%20it%20easy%20to,as%20markers%20on%20the%20map.) map created below allows users to generate a WKT by drawing instead of having to write it out by hand

In [ ]:
def download_image(url, output_dir, ann = False):
    """
    Downloads uavsar InSAR files from a url.
    Args:
        url (string): A url containing uavsar flight data. Can be from JPL or ASF
        output_dir (string): Directory to save the data in
    Returns:
        out_fp (string): File path to downloaded image.
    Raises:
       None
    """

    log.info(f'Starting download of {url}...')
    local = join(output_dir, basename(url))

    # Make the output dir if it doesn't exist
    if not isdir(output_dir):
        os.makedirs(output_dir)

    if not isfile(local):
        stream_download(url, local)
    else:
        log.info(f'{local} already exists, skipping download!')

    if ann:
        if url.split('.')[-1] == 'zip' or url.split('.')[-1] == 'ann':
            log.info('Download already contains ann file, skipping download!')
        else:
            parent = dirname(url)
            # ASF formatting - query parent directory
            if parent.split('.')[-1] == 'zip':
                log.debug(f'ASF url found for {url}')
                parent_files = requests.get(parent).json()['response']
                ann_info = [i for i in parent_files if '.ann' in i['name']][0]
                # assert len(ann_info) == 1, 'More than one ann file detected'
                ann_url = ann_info['url']
                log.debug(f'Annotation url: {ann_url}')

            # JPL formatting - have to parse url to get ann
            elif 'uavsar.asfdaac.alaska.edu' in url:
                log.debug(f'JPL url found for {url}')
                ext = url.split('.')[-1]
                pols = ['VVVV','HHHH','HVHV', 'HHHV', 'HHVV','HVVV']
                slc_pol = [pol for pol in pols if (pol in url)]
                if len(slc_pol) == 1:
                    url = url.replace(slc_pol[0], '')

                if ext == 'grd':
                    if len(basename(url).split('.')) == 2:
                        url = url.replace('.grd','.ann')
                    if len(basename(url).split('.')) == 3:
                        url = url.replace('.grd','')
                    ext = url.split('.')[-1]
                ann_url = url.replace(f'.{ext}', '.ann')
                log.debug(f'Annotation url: {ann_url}')

            else:
                log.warning('No ann url found. Unable to download ann file.')
                ann_url = None

            if ann_url:
                ann_local = join(output_dir, basename(ann_url))
                log.debug(f'Annotation local: {ann_local} and url {ann_url}')
                if not isfile(ann_local):
                    stream_download(ann_url, ann_local)
                else:
                    log.info(f'{ann_local} already exists, skipping download!')
    return local


In [111]:
# show map
m = Map()
draw = Draw(export = True, filename = 'asf_wkt.geojson')
draw.add_to(m)
m

In [113]:
wkt = gpd.read_file('/Users/jacktarricone/Downloads/asf_wkt (7).geojson').to_wkt().loc[0, 'geometry']

In [87]:
# define search parameters for sierra flight line
# asf_search uses WKT polygon 	POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))
# you need 5 coordinates so it connects back to the first point

#sierra_wkt = 'POLYGON((-119.9697 37.4631,-118.9576 37.4631,-118.9576 38.7211,-119.969 38211,-119.9697 37.4631))'
# wkt = 'POLYGON((-135.7 58.2,-136.6 58.1,-135.8 56.9,-134.6 56.1,-134.9 58.0,-135.7 58.2))'
start_date = datetime.strptime('2021-02-03 11:00:00', '%Y-%m-%d %H:%M:%S') 
end_date = datetime.strptime('2021-02-05 11:00:00', '%Y-%m-%d %H:%M:%S') 

## Seach for data
-use asf search, talk about all the parameters that can be used for search
-show different types of processing, state that GRD is important
-

In [114]:
# search for data
results = asf.search(platform = 'UAVSAR',
                     processingLevel = (['INTERFEROMETRY_GRD','AMPLITUDE_GRD']),
                     intersectsWith = wkt,
                     start = start_date,
                     end = end_date)

print(f'{len(results)} product(s) found')

6 product(s) found


In [93]:
# pull out three search results and test print the first one
first = results[0]
second = results[1]
third = results[2]
fourth = results[3]
fifth = results[4]
sixth = results[5]
print(first.properties)

{'beamModeType': 'RPI', 'browse': ['https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.hgt.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.hgt.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090VH_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090VH_01.hgt.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.hgt.png'], 'bytes': '5611958264', 'faradayRotation': None, 'fileID': 'UA_lowman_05208_21009-005_21012-004_0007d_s01_L090_01-INT

In [94]:
# define the URLs and print
first_url = first.properties['url']
second_url = second.properties['url']
third_url = third.properties['url']
fourth_url = fourth.properties['url']
fifth_url = fifth.properties['url']
sixth_url = sixth.properties['url']

print(first_url)
print(second_url)
print(third_url)
print(fourth_url)
print(fifth_url)
print(sixth_url)

https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090_01_int_grd.zip
https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/lowman_23205_21009-004_21012-000_0007d_s01_L090_01_int_grd.zip
https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/lowman_23205_21008-000_21009-004_0007d_s01_L090_01_int_grd.zip
https://datapool.asf.alaska.edu/AMPLITUDE_GRD/UA/lowman_05208_21009-005_21012-004_0007d_s01_L090_01_amp_grd.zip
https://datapool.asf.alaska.edu/AMPLITUDE_GRD/UA/lowman_23205_21009-004_21012-000_0007d_s01_L090_01_amp_grd.zip
https://datapool.asf.alaska.edu/AMPLITUDE_GRD/UA/lowman_23205_21008-000_21009-004_0007d_s01_L090_01_amp_grd.zip


In [115]:
# define outpout dir and list files
os.chdir('/Users/jacktarricone/Desktop/zach_test')
output_dir = '/Users/jacktarricone/Desktop/zach_test/'
files = os.listdir(output_dir) # print contents of output directory
print(files)

['lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.int.grd', '.DS_Store', 'lowman_05208_21009-005_21012-004_0007d_s01_L090VH_01.int.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.unw.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090VH_01.unw.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.cor.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.cor.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.hgt.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.hgt.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.int.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.int.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.ann', 'lowman_05208_21009-005_21012-004_0007d_s01_L090_01_int_grd.zip', 'lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.ann', 'lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.hgt.grd', 'lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.ann', 'lowman_05208_21009-005_21012-004_00

In [102]:
# download data
download_image(url = first_url, output_dir = output_dir)

'/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090_01_int_grd.zip'

In [108]:
zipped_data = glob.glob("*.zip") # print contents of output directory
print(zipped_data[0])

lowman_05208_21009-005_21012-004_0007d_s01_L090_01_int_grd.zip


In [110]:
with zipfile.ZipFile(zipped_data[0], "r") as zip_ref:
    zip_ref.printdir()
    print('Extracting all the files now...')
    zip_ref.extractall()
    print("done")

File Name                                             Modified             Size
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.cor.grd 2021-04-21 01:20:54   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.hgt.grd 2021-04-21 01:46:44   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.int.grd 2021-04-21 01:47:38   3672140080
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.unw.grd 2021-04-21 01:49:18   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.cor.grd 2021-04-21 01:50:06   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.hgt.grd 2021-04-21 01:50:56   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.int.grd 2021-04-21 01:51:50   3672140080
lowman_05208_21009-005_21012-004_0007d_s01_L090HV_01.unw.grd 2021-04-21 01:53:32   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090VH_01.cor.grd 2021-04-21 01:54:20   1836070040
lowman_05208_21009-005_21012-004_0007d_s01_L090VH_01.hgt.grd 2021-04-21 01

In [118]:
# search for unw and ann files for download test
for unw in glob.glob("*HH*.grd"):
    print(unw)
    
for ann in glob.glob("*HH*ann*"):
    print(ann)

lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.cor.grd
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.hgt.grd
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.int.grd
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.unw.grd
lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.ann


In [146]:
# define the HH GRD binary files and the annotation file
grd_list = glob.glob(os.path.abspath("*HH*.grd"))
ann = glob.glob(os.path.abspath("*HH*.ann"))
print(grd_list)
print(ann)

['/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.cor.grd', '/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.hgt.grd', '/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.int.grd', '/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.unw.grd']
['/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.ann']


In [148]:
print(ann)

['/Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.ann']


In [140]:
type(grd_list[0])

str

In [155]:
for file in grd_list:
    convert_image(in_fp = file,
                  ann_fp = ann[0],
                  out_fp = file + ".tif")

INFO:uavsar_pytools.convert:Working with insar
INFO:uavsar_pytools.convert:type = grd
DEBUG:uavsar_pytools.convert:rows: 17255 x cols: 26602 pixels
DEBUG:uavsar_pytools.convert:latitude delta: -5.556e-05, longitude delta: 5.556e-05 deg/pixel
DEBUG:uavsar_pytools.convert:Ref Latitude: 44.512116240000005, Longitude: -116.32247172000001 degrees
DEBUG:uavsar_pytools.convert:Bytes = 4, Endian = LITTLE ENDIAN
DEBUG:uavsar_pytools.convert:grd
DEBUG:uavsar_pytools.convert:Complex descriptor REAL*4
DEBUG:uavsar_pytools.convert:Data type = [('real', '<f4')]
DEBUG:uavsar_pytools.convert:Writing real component to /Users/jacktarricone/Desktop/zach_test/lowman_05208_21009-005_21012-004_0007d_s01_L090HH_01.cor.grd.tif...
INFO:uavsar_pytools.convert:Working with insar
INFO:uavsar_pytools.convert:type = grd
DEBUG:uavsar_pytools.convert:rows: 17255 x cols: 26602 pixels
DEBUG:uavsar_pytools.convert:latitude delta: -5.556e-05, longitude delta: 5.556e-05 deg/pixel
DEBUG:uavsar_pytools.convert:Ref Latitude:

In [ ]:
#### this works once i changed everything to epsg:4326